# Career Exploration Final Project: TMDB Box Office Prediction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## Data Loading

In [2]:
train = pd.read_csv('data/train.csv')

In [3]:
test = pd.read_csv('data/test.csv')

In [4]:
train.loc[train['id'] == 16,'revenue'] = 192864          # Skinning
train.loc[train['id'] == 90,'budget'] = 30000000         # Sommersby          
train.loc[train['id'] == 118,'budget'] = 60000000        # Wild Hogs
train.loc[train['id'] == 149,'budget'] = 18000000        # Beethoven
train.loc[train['id'] == 313,'revenue'] = 12000000       # The Cookout 
train.loc[train['id'] == 451,'revenue'] = 12000000       # Chasing Liberty
train.loc[train['id'] == 464,'budget'] = 20000000        # Parenthood
train.loc[train['id'] == 470,'budget'] = 13000000        # The Karate Kid, Part II
train.loc[train['id'] == 513,'budget'] = 930000          # From Prada to Nada
train.loc[train['id'] == 797,'budget'] = 8000000         # Welcome to Dongmakgol
train.loc[train['id'] == 819,'budget'] = 90000000        # Alvin and the Chipmunks: The Road Chip
train.loc[train['id'] == 850,'budget'] = 90000000        # Modern Times
train.loc[train['id'] == 1007,'budget'] = 2              # Zyzzyx Road 
train.loc[train['id'] == 1112,'budget'] = 7500000        # An Officer and a Gentleman
train.loc[train['id'] == 1131,'budget'] = 4300000        # Smokey and the Bandit   
train.loc[train['id'] == 1359,'budget'] = 10000000       # Stir Crazy 
train.loc[train['id'] == 1542,'budget'] = 1              # All at Once
train.loc[train['id'] == 1570,'budget'] = 15800000       # Crocodile Dundee II
train.loc[train['id'] == 1571,'budget'] = 4000000        # Lady and the Tramp
train.loc[train['id'] == 1714,'budget'] = 46000000       # The Recruit
train.loc[train['id'] == 1721,'budget'] = 17500000       # Cocoon
train.loc[train['id'] == 1865,'revenue'] = 25000000      # Scooby-Doo 2: Monsters Unleashed
train.loc[train['id'] == 1885,'budget'] = 12             # In the Cut
train.loc[train['id'] == 2091,'budget'] = 10             # Deadfall
train.loc[train['id'] == 2268,'budget'] = 17500000       # Madea Goes to Jail budget
train.loc[train['id'] == 2491,'budget'] = 6              # Never Talk to Strangers
train.loc[train['id'] == 2602,'budget'] = 31000000       # Mr. Holland's Opus
train.loc[train['id'] == 2612,'budget'] = 15000000       # Field of Dreams
train.loc[train['id'] == 2696,'budget'] = 10000000       # Nurse 3-D
train.loc[train['id'] == 2801,'budget'] = 10000000       # Fracture
train.loc[train['id'] == 335,'budget'] = 2 
train.loc[train['id'] == 348,'budget'] = 12
train.loc[train['id'] == 470,'budget'] = 13000000 
train.loc[train['id'] == 513,'budget'] = 1100000
train.loc[train['id'] == 640,'budget'] = 6 
train.loc[train['id'] == 696,'budget'] = 1
train.loc[train['id'] == 797,'budget'] = 8000000 
train.loc[train['id'] == 850,'budget'] = 1500000
train.loc[train['id'] == 1199,'budget'] = 5 
train.loc[train['id'] == 1282,'budget'] = 9               # Death at a Funeral
train.loc[train['id'] == 1347,'budget'] = 1
train.loc[train['id'] == 1755,'budget'] = 2
train.loc[train['id'] == 1801,'budget'] = 5
train.loc[train['id'] == 1918,'budget'] = 592 
train.loc[train['id'] == 2033,'budget'] = 4
train.loc[train['id'] == 2118,'budget'] = 344 
train.loc[train['id'] == 2252,'budget'] = 130
train.loc[train['id'] == 2256,'budget'] = 1 
train.loc[train['id'] == 2696,'budget'] = 10000000

#Clean Data
test.loc[test['id'] == 6733,'budget'] = 5000000
test.loc[test['id'] == 3889,'budget'] = 15000000
test.loc[test['id'] == 6683,'budget'] = 50000000
test.loc[test['id'] == 5704,'budget'] = 4300000
test.loc[test['id'] == 6109,'budget'] = 281756
test.loc[test['id'] == 7242,'budget'] = 10000000
test.loc[test['id'] == 7021,'budget'] = 17540562       #  Two Is a Family
test.loc[test['id'] == 5591,'budget'] = 4000000        # The Orphanage
test.loc[test['id'] == 4282,'budget'] = 20000000       # Big Top Pee-wee
test.loc[test['id'] == 3033,'budget'] = 250 
test.loc[test['id'] == 3051,'budget'] = 50
test.loc[test['id'] == 3084,'budget'] = 337
test.loc[test['id'] == 3224,'budget'] = 4  
test.loc[test['id'] == 3594,'budget'] = 25  
test.loc[test['id'] == 3619,'budget'] = 500  
test.loc[test['id'] == 3831,'budget'] = 3  
test.loc[test['id'] == 3935,'budget'] = 500  
test.loc[test['id'] == 4049,'budget'] = 995946 
test.loc[test['id'] == 4424,'budget'] = 3  
test.loc[test['id'] == 4460,'budget'] = 8  
test.loc[test['id'] == 4555,'budget'] = 1200000 
test.loc[test['id'] == 4624,'budget'] = 30 
test.loc[test['id'] == 4645,'budget'] = 500 
test.loc[test['id'] == 4709,'budget'] = 450 
test.loc[test['id'] == 4839,'budget'] = 7
test.loc[test['id'] == 3125,'budget'] = 25 
test.loc[test['id'] == 3142,'budget'] = 1
test.loc[test['id'] == 3201,'budget'] = 450
test.loc[test['id'] == 3222,'budget'] = 6
test.loc[test['id'] == 3545,'budget'] = 38
test.loc[test['id'] == 3670,'budget'] = 18
test.loc[test['id'] == 3792,'budget'] = 19
test.loc[test['id'] == 3881,'budget'] = 7
test.loc[test['id'] == 3969,'budget'] = 400
test.loc[test['id'] == 4196,'budget'] = 6
test.loc[test['id'] == 4221,'budget'] = 11
test.loc[test['id'] == 4222,'budget'] = 500
test.loc[test['id'] == 4285,'budget'] = 11
test.loc[test['id'] == 4319,'budget'] = 1
test.loc[test['id'] == 4639,'budget'] = 10
test.loc[test['id'] == 4719,'budget'] = 45
test.loc[test['id'] == 4822,'budget'] = 22
test.loc[test['id'] == 4829,'budget'] = 20
test.loc[test['id'] == 4969,'budget'] = 20
test.loc[test['id'] == 5021,'budget'] = 40 
test.loc[test['id'] == 5035,'budget'] = 1 
test.loc[test['id'] == 5063,'budget'] = 14 
test.loc[test['id'] == 5119,'budget'] = 2 
test.loc[test['id'] == 5214,'budget'] = 30 
test.loc[test['id'] == 5221,'budget'] = 50 
test.loc[test['id'] == 4903,'budget'] = 15
test.loc[test['id'] == 4983,'budget'] = 3
test.loc[test['id'] == 5102,'budget'] = 28
test.loc[test['id'] == 5217,'budget'] = 75
test.loc[test['id'] == 5224,'budget'] = 3 
test.loc[test['id'] == 5469,'budget'] = 20 
test.loc[test['id'] == 5840,'budget'] = 1 
test.loc[test['id'] == 5960,'budget'] = 30
test.loc[test['id'] == 6506,'budget'] = 11 
test.loc[test['id'] == 6553,'budget'] = 280
test.loc[test['id'] == 6561,'budget'] = 7
test.loc[test['id'] == 6582,'budget'] = 218
test.loc[test['id'] == 6638,'budget'] = 5
test.loc[test['id'] == 6749,'budget'] = 8 
test.loc[test['id'] == 6759,'budget'] = 50 
test.loc[test['id'] == 6856,'budget'] = 10
test.loc[test['id'] == 6858,'budget'] =  100
test.loc[test['id'] == 6876,'budget'] =  250
test.loc[test['id'] == 6972,'budget'] = 1
test.loc[test['id'] == 7079,'budget'] = 8000000
test.loc[test['id'] == 7150,'budget'] = 118
test.loc[test['id'] == 6506,'budget'] = 118
test.loc[test['id'] == 7225,'budget'] = 6
test.loc[test['id'] == 7231,'budget'] = 85
test.loc[test['id'] == 5222,'budget'] = 5
test.loc[test['id'] == 5322,'budget'] = 90
test.loc[test['id'] == 5350,'budget'] = 70
test.loc[test['id'] == 5378,'budget'] = 10
test.loc[test['id'] == 5545,'budget'] = 80
test.loc[test['id'] == 5810,'budget'] = 8
test.loc[test['id'] == 5926,'budget'] = 300
test.loc[test['id'] == 5927,'budget'] = 4
test.loc[test['id'] == 5986,'budget'] = 1
test.loc[test['id'] == 6053,'budget'] = 20
test.loc[test['id'] == 6104,'budget'] = 1
test.loc[test['id'] == 6130,'budget'] = 30
test.loc[test['id'] == 6301,'budget'] = 150
test.loc[test['id'] == 6276,'budget'] = 100
test.loc[test['id'] == 6473,'budget'] = 100
test.loc[test['id'] == 6842,'budget'] = 30

In [5]:
X_train, y_train = train.drop('revenue', axis=1), train['revenue']
X_test = test

In [6]:
df = pd.concat((X_train, X_test), axis=0)#, sort=False)

## Exploratory Data Analysis

In [7]:
def empty_listify(lst):
    return [] if pd.isnull(lst) else eval(lst)

def json_to_list_of_ids(col):
    return pd.Series(
        map(
            lambda lst: list(col['id'] for col in empty_listify(lst)), 
            col
        )
    )

In [8]:
from sklearn.decomposition import PCA
from collections import defaultdict

def pcaify(one_hot, column_prefix, num_pca_columns):
    pca = PCA(n_components=num_pca_columns)    
    features = pca.fit_transform(one_hot)
    
    return pd.DataFrame(data = features, columns = ['{0}_{1}'.format(column_prefix, i) for i in range(features.shape[1])])

def one_hot_encode_json_col(col, prefix, num_columns, num_pca_columns):
    dummies = []

    counts = defaultdict(int)
    
    for entry in col:
        for elem in empty_listify(entry):
            counts[elem['name']] += 1
    
    top = {k for k, v in sorted(counts.items(), key=lambda item: -counts[item[0]])[:num_columns]}
    
    for entry in col:
        d = {}
        for elem in empty_listify(entry):
            if elem['name'] in top:
                d[f'{prefix}_{elem["name"]}'] = 1
        dummies.append(d)
    
    if len(top) < num_pca_columns:
        return pd.DataFrame(dummies).fillna(0).set_index(df.index)
    
    one_hot = pd.DataFrame(dummies).fillna(0).values
    
    return pcaify(one_hot, prefix, num_pca_columns).set_index(df.index)

In [9]:
df = pd.concat((df, one_hot_encode_json_col(df['genres'], 'genre', num_columns=100, num_pca_columns=30)), axis=1).drop('genres', axis=1)

In [10]:
df = pd.concat((df, one_hot_encode_json_col(df['production_companies'], 'pc', num_columns=100, num_pca_columns=10)), axis=1).drop('production_companies', axis=1)

In [11]:
df = pd.concat((df, one_hot_encode_json_col(df['Keywords'], 'kw', num_columns=1000, num_pca_columns=50)), axis=1).drop('Keywords', axis=1)

### Text Data

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import HashingVectorizer

def remove_stopwords(sentence):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence) 
    return ' '.join([w for w in word_tokens if not w in stop_words])

def get_text_features(df, text_col, num_text_features=10):
    print('Removing stopwords')
    text = df[text_col].fillna('').apply(remove_stopwords)

    print('Creating bag of words')
    vectorizer = HashingVectorizer()
    text_bow = vectorizer.fit_transform(text).toarray()
    
    print('Running PCA')
    pca = PCA(n_components=num_text_features)
    text_features = pca.fit_transform(text_bow)
    
    return pd.DataFrame(data = text_features, columns = ['{0}_{1}'.format(text_col, i) for i in range(text_features.shape[1])])

In [13]:
text_data = df[['overview', 'tagline', 'original_title']]

In [14]:
df['has_tagline'] = df['tagline'].isnull().astype(int)
df = df.drop('tagline', axis=1)

### Release Date

In [15]:
release_date = pd.to_datetime(df['release_date'])

In [16]:
release_date[pd.isnull(release_date)] = pd.to_datetime((release_date - release_date.min()).mean() + release_date.min())

In [17]:
df['release_date_days'] = (release_date - release_date.min()).apply(lambda d: d.days)

In [18]:
df['release_date_month'] = release_date.apply(lambda d: d.month)

In [19]:
df['release_date_year'] = release_date.apply(lambda d: d.year)

#### Adjusting for year

In [20]:
# df['meanruntimeByYear'] = df.groupby("release_date_year")["runtime"].aggregate('mean')
# df['meanPopularityByYear'] = df.groupby("release_date_year")["popularity"].aggregate('mean')
# df['meanBudgetByYear'] = df.groupby("release_date_year")["budget"].aggregate('mean')
# df['medianBudgetByYear'] = df.groupby("release_date_year")["budget"].aggregate('median')

In [21]:
df = df.drop('release_date', axis=1)

### Cast

In [22]:
cast = df['cast']

In [23]:
df = pd.concat((df, one_hot_encode_json_col(cast, 'cast', num_columns=1000, num_pca_columns=100)), axis=1).drop('cast', axis=1)

In [24]:
cast_male_counts = []
cast_female_counts = []

for entry in cast:
    entry_data = empty_listify(entry)
    if entry_data:
        cast_male_counts.append(
            sum(1 for e in entry_data if e['gender'] == 2)
        )
        cast_female_counts.append(
            sum(1 for e in entry_data if e['gender'] == 1)
        )

In [25]:
df['num_male'] = pd.Series(cast_male_counts)
df['num_female'] = pd.Series(cast_female_counts)

#### Crew

In [26]:
crew = df['crew']

In [27]:
crew_counts = []

for entry in crew:
    entry_data = empty_listify(entry)
    if entry_data:
        crew_counts.append(len(entry_data))

In [28]:
df['num_crew'] = pd.Series(crew_counts)

In [29]:
df = df.drop('crew', axis=1)

### Other Categories

#### Original Language

In [30]:
df = pd.concat((df, pcaify(pd.get_dummies(df['original_language']), 'og_lang', 20).set_index(df.index)), axis=1)
df = df.drop('original_language', axis=1)

#### Spoken Languages

In [31]:
df = pd.concat((df, one_hot_encode_json_col(df['spoken_languages'], 'sl', num_columns=500, num_pca_columns=10)), axis=1).drop('spoken_languages', axis=1)

#### Homepage

In [32]:
df['has_homepage'] = df['homepage'].isnull().astype(int)
df = df.drop('homepage', axis=1)

#### Production Countries

In [33]:
df = pd.concat((df, one_hot_encode_json_col(df['production_countries'], 'sl', num_columns=500, num_pca_columns=10)), axis=1).drop('production_countries', axis=1)

#### Status

In [34]:
df['is_released'] = (df['status'] == 'Released').astype(int)
df = df.drop('status', axis=1)

### Cleanup

In [35]:
df = df.drop(['status', 'title', 'id', 'belongs_to_collection', 'original_title', 'imdb_id', 'overview', 'poster_path', 'original_title'], axis=1)

In [36]:
df['runtime'] = df['runtime'].fillna(df['runtime'].mean())

In [37]:
from sklearn.preprocessing import StandardScaler

df = pd.DataFrame(data = StandardScaler().fit_transform(df), columns = df.columns)

In [38]:
X = df#[['budget', 'popularity', 'runtime', 'release_date_days', 'release_date_month', 'has_homepage', 'has_tagline']]

## Validation

In [39]:
from sklearn.metrics import mean_squared_log_error

def evaluate(y_pred, y_true):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [40]:
from sklearn.model_selection import train_test_split

X_train = X[:train.shape[0]]
y_train = y_train
X_test = X[train.shape[0]:]

train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [41]:
def predict(model, X):
    non_transformed = model.predict(X)
    non_transformed = np.clip(non_transformed, 0, non_transformed.max())
    return np.exp(non_transformed)

### Linear Regression

In [42]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [43]:
model = Lasso(alpha=0.05)

In [44]:
model.fit(train_X, np.log(train_y))

Lasso(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [45]:
evaluate(predict(model, valid_X), valid_y)

2.231733871433978

In [46]:
evaluate(predict(model, train_X), train_y)

2.329478029220208

### Random Forest

In [47]:
from sklearn.ensemble import RandomForestRegressor

/Users/ajay/anaconda3/envs/deeplearning/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [48]:
model = RandomForestRegressor(max_depth=8, random_state=8, n_estimators=100)

In [49]:
model.fit(train_X, np.log(train_y))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=8, verbose=0, warm_start=False)

In [50]:
evaluate(predict(model, train_X), train_y)

1.4663611741259006

In [51]:
evaluate(predict(model, valid_X), valid_y)

1.95752059850231

### KNN

In [52]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_neighbors=20)
model.fit(train_X, np.log(train_y))
y_pred = model.predict(valid_X)

In [53]:
evaluate(predict(model, valid_X), valid_y)

3.092278932332581

### Neural Network

In [60]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [61]:
model = Sequential()
model.add(Dense(train_X.shape[1], activation='relu', input_shape=(train_X.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu', input_shape=(64,)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', input_shape=(32,)))
model.add(Dense(1, activation='relu'))

In [62]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(optimizer = "sgd", loss = root_mean_squared_error)

In [63]:
model.fit(x=train_X.values, y=np.log(train_y.values), epochs=200)

Epoch 1/200
2400/2400 [==============================] - 0s 201us/step - loss: 10.5744
Epoch 2/200
2400/2400 [==============================] - 0s 68us/step - loss: 6.0602
Epoch 3/200
2400/2400 [==============================] - 0s 64us/step - loss: 5.2016
Epoch 4/200
2400/2400 [==============================] - 0s 68us/step - loss: 4.8252
Epoch 5/200
2400/2400 [==============================] - 0s 67us/step - loss: 4.4466
Epoch 6/200
2400/2400 [==============================] - 0s 69us/step - loss: 4.1408
Epoch 7/200
2400/2400 [==============================] - 0s 63us/step - loss: 4.0487
Epoch 8/200
2400/2400 [==============================] - 0s 66us/step - loss: 3.8179: 0s - loss: 3.825
Epoch 9/200
2400/2400 [==============================] - 0s 67us/step - loss: 3.6143
Epoch 10/200
2400/2400 [==============================] - 0s 65us/step - loss: 3.4879
Epoch 11/200
2400/2400 [==============================] - 0s 64us/step - loss: 3.3094
Epoch 12/200
2400/2400 [===================

2400/2400 [==============================] - 0s 67us/step - loss: 2.1438
Epoch 97/200
2400/2400 [==============================] - 0s 64us/step - loss: 2.1127
Epoch 98/200
2400/2400 [==============================] - 0s 64us/step - loss: 2.0886
Epoch 99/200
2400/2400 [==============================] - 0s 74us/step - loss: 2.0810
Epoch 100/200
2400/2400 [==============================] - 0s 70us/step - loss: 2.0585
Epoch 101/200
2400/2400 [==============================] - 0s 71us/step - loss: 2.0543
Epoch 102/200
2400/2400 [==============================] - 0s 63us/step - loss: 2.0115
Epoch 103/200
2400/2400 [==============================] - 0s 65us/step - loss: 1.9837
Epoch 104/200
2400/2400 [==============================] - 0s 67us/step - loss: 1.9949
Epoch 105/200
2400/2400 [==============================] - 0s 69us/step - loss: 1.9964
Epoch 106/200
2400/2400 [==============================] - 0s 68us/step - loss: 2.0199
Epoch 107/200
2400/2400 [==============================] - 0

In [64]:
evaluate(predict(model, valid_X.values), valid_y)

3.361275334834239

In [65]:
evaluate(predict(model, train_X.values), train_y)

2.7037621572063717

### XGBoost

In [54]:
from sklearn.model_selection import KFold

random_seed = 42
k = 10
fold = list(KFold(k, shuffle = True, random_state = random_seed).split(X_train.values))
np.random.seed(random_seed)

In [55]:
import xgboost as xgb
from datetime import datetime

def xgb_model(trn_x, trn_y, val_x, val_y, test, verbose) :
    
    params = {'objective': 'reg:linear', 
              'eta': 0.01, 
              'max_depth': 6, 
              'subsample': 0.6, 
              'colsample_bytree': 0.7,  
              'eval_metric': 'rmse', 
              'seed': random_seed, 
              'silent': True,
    }
    
    record = dict()
    model = xgb.train(params
                      , xgb.DMatrix(trn_x, trn_y)
                      , 100000
                      , [(xgb.DMatrix(trn_x, trn_y), 'train'), (xgb.DMatrix(val_x, val_y), 'valid')]
                      , verbose_eval=verbose
                      , early_stopping_rounds=500
                      , callbacks = [xgb.callback.record_evaluation(record)])
    best_idx = np.argmin(np.array(record['valid']['rmse']))

    val_pred = model.predict(xgb.DMatrix(val_x), ntree_limit=model.best_ntree_limit)
    test_pred = model.predict(xgb.DMatrix(X_test.values), ntree_limit=model.best_ntree_limit)

    return {'val':val_pred, 'test':test_pred, 'error':record['valid']['rmse'][best_idx], 'importance':[i for k, i in model.get_score().items()]}

In [61]:
def run_xgb(X_train, y_train, X_test):
    result_dict = dict()
    val_pred = np.zeros(X_train.values.shape[0])
    test_pred = np.zeros(X_test.shape[0])
    final_err = 0
    verbose = False

    for i, (trn, val) in enumerate(fold) :
        print(i+1, "fold.    RMSE")

        trn_x = X_train.values[trn, :]
        trn_y = y_train[trn]
        val_x = X_train.values[val, :]
        val_y = y_train[val]

        fold_val_pred = []
        fold_test_pred = []
        fold_err = []

        #""" xgboost
        start = datetime.now()
        result = xgb_model(trn_x, trn_y, val_x, val_y, X_test, verbose)
        fold_val_pred.append(result['val'])
        fold_test_pred.append(result['test'])
        fold_err.append(result['error'])
        print("xgb model.", "{0:.5f}".format(result['error']), '(' + str(int((datetime.now()-start).seconds/60)) + 'm)')
        #"""

        val_pred[val] += np.mean(np.array(fold_val_pred), axis = 0)
        test_pred += np.mean(np.array(fold_test_pred), axis = 0) / k
        final_err += (sum(fold_err) / len(fold_err)) / k

    print("fianl avg   err.", final_err)
    print("fianl blend err.", np.sqrt(np.mean((val_pred - y_train)**2)))
    
    return test_pred

In [62]:
xgb_preds = run_xgb(X_train, np.log(y_train), X_test)

1 fold.    RMSE
xgb model. 1.91630 (0m)
2 fold.    RMSE
xgb model. 1.94071 (0m)
3 fold.    RMSE
xgb model. 2.04864 (0m)
4 fold.    RMSE
xgb model. 1.92870 (0m)
5 fold.    RMSE
xgb model. 2.17795 (0m)
6 fold.    RMSE
xgb model. 1.85512 (0m)
7 fold.    RMSE
xgb model. 2.33851 (0m)
8 fold.    RMSE
xgb model. 2.27653 (2m)
9 fold.    RMSE
xgb model. 2.24241 (0m)
10 fold.    RMSE
xgb model. 2.05955 (0m)
fianl avg   err. 2.0784402
fianl blend err. 2.0847448735154424


## Prediction

In [64]:
# preds = predict(model, X_test).reshape(-1)
preds = np.exp(xgb_preds)

In [65]:
out = pd.DataFrame(data={'id': test['id'], 'revenue': preds}).set_index('id')

In [66]:
out.to_csv('submission.csv')